
# TP 1 — Cartographier un dataset (RGPD)
**Objectif :** Identifier les colonnes personnelles / sensibles / non‑identifiantes, puis proposer des mesures d’anonymisation ou de pseudonymisation.  
**Livrable :** un tableau *colonne → catégorie → mesure de protection* + un dataset transformé.



> **Consigne** : Seule la génération du dataset est fournie en entier.  
> Pour le reste, vous avez des **exemples de code** et des **TODO** à compléter.


## Étapes
1. Générer un dataset.
2. Inspecter les colonnes et inférer une **catégorie** proposée (heuristique).
3. Ajuster/valider la **catégorie** manuellement.
4. Choisir une **mesure de protection** par colonne (suppression, hachage, généralisation, bruit, etc.).
5. Appliquer la transformation et **exporter** : livrable (.csv) + dataset transformé.


## 1) Génération du dataset
Ce bloc génère un dataset synthétique réaliste (400 lignes). **Exécutez-le tel quel.**

### Installation des bibliothéque nécessaires

In [1]:
%pip install numpy pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

def generate_demo_dataset(n=400, seed=42):
    rng = np.random.default_rng(seed)
    first_names_m = ["Lucas","Hugo","Arthur","Liam","Raphaël","Gabin","Jules","Ethan","Noah","Louis"]
    first_names_f = ["Emma","Jade","Louise","Alice","Chloé","Lina","Mia","Rose","Anna","Léa"]
    last_names = ["Martin","Bernard","Dubois","Thomas","Robert","Richard","Petit","Durand","Leroy","Moreau"]
    countries = ["FR","DE","IT","ES","US"]
    cities = ["Paris","Lyon","Marseille","Berlin","Rome","Madrid","NYC"]
    salary_bands = ["<30k","30-60k","60-90k",">90k"]
    health = ["none","diabetes","hypertension","depression"]

    genders = rng.choice(["male","female"], size=n)
    first_names = [rng.choice(first_names_m if g=="male" else first_names_f) for g in genders]
    last = rng.choice(last_names, size=n)
    full_name = [f"{fn} {ln}" for fn, ln in zip(first_names, last)]
    email = [re.sub(r'[^a-z]','', fn.lower()) + "." + re.sub(r'[^a-z]','', ln.lower()) + f"{rng.integers(1,9999)}@example.com" for fn, ln in zip(first_names, last)]
    phone = [f"+33{rng.integers(600000000, 799999999)}" for _ in range(n)]
    age = rng.integers(16, 85, size=n)
    country = rng.choice(countries, size=n, p=[0.4,0.2,0.15,0.15,0.1])
    city = rng.choice(cities, size=n)
    salary_band = rng.choice(salary_bands, size=n, p=[0.35,0.4,0.2,0.05])
    ip_address = [f"192.168.{rng.integers(0,255)}.{rng.integers(1,255)}" for _ in range(n)]
    gps_lat = 48.85 + rng.normal(0, 0.05, size=n)
    gps_lon = 2.35 + rng.normal(0, 0.08, size=n)
    health_condition = rng.choice(health, size=n, p=[0.7,0.1,0.15,0.05])
    purchase_amount = np.round(rng.normal(120, 40, size=n).clip(min=0), 2)
    churned = rng.choice([0,1], size=n, p=[0.8,0.2])

    df_demo = pd.DataFrame({
        "id": np.arange(1, n+1),
        "full_name": full_name,
        "email": email,
        "phone": phone,
        "age": age,
        "gender": genders,
        "country": country,
        "city": city,
        "salary_band": salary_band,
        "ip_address": ip_address,
        "gps_lat": gps_lat,
        "gps_lon": gps_lon,
        "health_condition": health_condition,
        "purchase_amount": purchase_amount,
        "churned": churned,
    })
    return df_demo

df = generate_demo_dataset()
df.head() # pour lire les cinq premières lignes du dataframe

,id,full_name,email,phone,age,gender,country,city,salary_band,ip_address,gps_lat,gps_lon,health_condition,purchase_amount,churned
0,1,Louis Martin,louis.martin8789@example.com,+33660300048,43,male,IT,Paris,<30k,192.168.174.85,48.951032,2.383797,none,181.78,0
1,2,Rose Thomas,rose.thomas4775@example.com,+33747716076,26,female,DE,Marseille,30-60k,192.168.183.125,48.763251,2.408082,none,93.08,0
2,3,Alice Richard,alice.richard9846@example.com,+33765190062,79,female,FR,NYC,30-60k,192.168.112.120,48.874989,2.443542,none,101.04,0
3,4,Louis Moreau,louis.moreau7613@example.com,+33663430633,60,male,DE,Berlin,<30k,192.168.39.182,48.766044,2.256426,hypertension,56.50,0
4,5,Raphaël Durand,raphal.durand6508@example.com,+33624835381,38,male,ES,Lyon,<30k,192.168.26.184,48.844588,2.410389,hypertension,89.70,1


## 2) Exploration (à compléter)
- Aperçu (`head()`), types (`dtypes`), NA (`isna().sum()`), cardinalités (`nunique()`)

**En quoi ces informations pourraient nous-être utiles dans le cadre du RGPD ?**

In [4]:
# EXEMPLE (+ TODO)
# display(df.head(10)) # pour lire les dix premières lignes du dataframe
# df.dtypes # pour voir les types de données de chaque colonne
# df.isna().sum() # pour compter les valeurs manquantes par colonne
# df.nunique() # pour compter le nombre de valeurs uniques par colonne

print(f"Les 10 premières lignes du dataframe:\n{df.head(10).to_string(index=False)}")
print("-"*80)
print(f"\nLes types de données de chaque colonne:\n{df.dtypes.to_string()}")
print("-"*80)
print(f"\nLe nombre de valeurs manquantes par colonne:\n{df.isna().sum().to_string()}")
print("-"*80)
print(f"\nLe nombre de valeurs uniques par colonne:\n{df.nunique().to_string()}")

Les 10 premières lignes du dataframe:
 id       full_name                          email        phone  age gender country      city salary_band      ip_address   gps_lat  gps_lon health_condition  purchase_amount  churned
  1    Louis Martin   louis.martin8789@example.com +33660300048   43   male      IT     Paris        <30k  192.168.174.85 48.951032 2.383797             none           181.78        0
  2     Rose Thomas    rose.thomas4775@example.com +33747716076   26 female      DE Marseille      30-60k 192.168.183.125 48.763251 2.408082             none            93.08        0
  3   Alice Richard  alice.richard9846@example.com +33765190062   79 female      FR       NYC      30-60k 192.168.112.120 48.874989 2.443542             none           101.04        0
  4    Louis Moreau   louis.moreau7613@example.com +33663430633   60   male      DE    Berlin        <30k  192.168.39.182 48.766044 2.256426     hypertension            56.50        0
  5  Raphaël Durand  raphal.durand6508@exa

## 3) Classification des colonnes (à compléter)

Classification initiale (heuristique) : *personnelle / sensibles / non-identifiantes*
- Basée sur des **mots-clés** dans les noms de colonnes et sur la **cardinalité**.
- Vous pourrez **modifier** la catégorie ensuite.


In [5]:
# les champs du dataframee en question : Les 10 premières lignes du dataframe:
#  id       full_name                          email        phone  age gender country      city salary_band      ip_address   gps_lat  gps_lon health_condition  purchase_amount  churned

PII_KEYWORDS = ["full_name", "email", "age", "gender","phone", "address", "gps_lat", "gps_lon", "city"]  # les champs considérés comme des données personnelles
SENSITIVE_KEYWORDS = ["ip_address", "health_condition", "salary_band"] # les champs considérés comme des données sensibles

def classify_column(colname, series) -> str:
    col = colname.lower()
    if any(k in col for k in SENSITIVE_KEYWORDS):
        return "sensibles"
    if any(k in col for k in PII_KEYWORDS):
        return "personnelle"

    # TODO : option -> utiliser la cardinalité pour repérer des quasi-identifiants
    # Définition :
    # Un quasi-identifiant (QI) est une donnée qui ne permet pas
    # d'identifier directement une personne, mais qui, combinée
    # à d'autres colonnes, peut le faire (ex : âge + ville + genre).
    #
    # Formule :
    # unique_ratio = nombre de valeurs uniques / nombre total de lignes
    #
    # Exemple :
    # unique_ratio = series.nunique(dropna=True) / len(series)
    #
    # Interprétation :
    # - Si unique_ratio > 0.9 → la colonne est très discriminante
    #   (presque une valeur unique par individu) → potentiellement un QI.
    # - Sinon → colonne plutôt agrégée → peu de risque d'identification.
    
    unique_ratio = series.nunique(dropna=True) / len(series)
    if unique_ratio > 0.9:
        return "quasi-identifiantes"
    return "non-identifiantes"

classification = []
for c in df.columns:
    cat = classify_column(c, df[c])
    classification.append({"column": c, "proposed_category": cat})

classif_df = pd.DataFrame(classification)
classif_df

,column,proposed_category
0,id,quasi-identifiantes
1,full_name,personnelle
2,email,personnelle
3,phone,personnelle
4,age,personnelle
5,gender,personnelle
6,country,non-identifiantes
7,city,personnelle
8,salary_band,sensibles
9,ip_address,sensibles


## 4) Ajuster la catégorie (manuel)
Éditez ci-dessous le dictionnaire `CATEGORY_OVERRIDE` si nécessaire (clé = nom de colonne, valeur parmi :  
`"personnelle"`, `"sensibles"`, `"quasi-identifiantes"`, `"non-identifiantes"`).


In [6]:
# Exemple d'override : décommentez et ajustez
CATEGORY_OVERRIDE = {
  "id": "non-identifiantes",
  "full_name": "personnelle",
  "health_condition": "sensibles",
  "email": "personnelle",
  "phone": "personnelle",
  "age": "personnelle",
  "gender": "personnelle",
  "country": "personnelle",
  "city": "personnelle",
  "salary_band": "sensibles",
  "ip_address": "sensibles",
   "gps_lat": "personnelle",
   "gps_lon": "personnelle",
   "health_condition": "sensibles",
   "purchase_amount": "non-identifiantes",
   "churned": "non-identifiantes",
}

def apply_overrides(classif_df, overrides: dict) -> pd.DataFrame:
    classif_df = classif_df.copy()
    classif_df["final_category"] = classif_df["proposed_category"]
    for k, v in overrides.items():
        if k in classif_df["column"].values:
            classif_df.loc[classif_df["column"] == k, "final_category"] = v
    return classif_df

classif_final = apply_overrides(classif_df, CATEGORY_OVERRIDE)
classif_final

,column,proposed_category,final_category
0,id,quasi-identifiantes,non-identifiantes
1,full_name,personnelle,personnelle
2,email,personnelle,personnelle
3,phone,personnelle,personnelle
4,age,personnelle,personnelle
5,gender,personnelle,personnelle
6,country,non-identifiantes,personnelle
7,city,personnelle,personnelle
8,salary_band,sensibles,sensibles
9,ip_address,sensibles,sensibles


## 5) Choisir la mesure de protection par colonne (A compléter)
Voici un **catalogue** de mesures. Vous pouvez définir un **plan** ci-dessous.

- `drop` : suppression pure et simple de la colonne
- `hash` : hachage SHA-256 (avec sel) → **pseudonymisation** (réversible si le sel est conservé)
- `bin` : généralisation/agrégation (discrétisation) pour variables numériques
- `mask_email` : masquage simple des emails (préservation du domaine)
- `truncate_phone` : masquage simple des téléphones
- `noise_num` : ajout d’un bruit aléatoire léger (⚠️ pas une DP formelle ligne‑à‑ligne)
- `keep` : conserver tel quel (si non‑identifiante)

> Pour une **Differential Privacy** rigoureuse, appliquez du bruit sur des **agrégats** (comptes, moyennes) avec un mécanisme adapté (Laplace/Gauss) — hors scope de ce mini‑TP.


In [ ]:
import hashlib

def sha256_hash_series(s: pd.Series, salt: str = "CHANGE_ME_SALT") -> pd.Series:
    # chash chaque valeur de la série avec SHA-256 et un sel
    def _h(x):
        if pd.isna(x): return x
        return hashlib.sha256((str(x) + salt).encode("utf-8")).hexdigest()
    return s.apply(_h)

def bin_numeric(s: pd.Series, bins=5) -> pd.Series:
    # Divise les valeurs numériques en 'bins' catégories égales
    return pd.qcut(s, q=bins, duplicates="drop")

def noise_numeric(s: pd.Series, scale=0.5) -> pd.Series:
    # Ajoute du bruit gaussien aux valeurs numériques
    noise = np.random.normal(0.0, scale, size=len(s))
    return (pd.to_numeric(s, errors="coerce") + noise).round(3)

plan_protection = {
    "id": "drop",
    "full_name": "drop",
    "email": "hash",
    # TODO : compléter pour chaque colonne
    "phone": "hash", 
    "age": "bin",
    "gender": "drop",
    "country": "drop",
    "city": "drop",
    "salary_band": "drop",
    "ip_address": "hash",
    "gps_lat": "noise",
    "gps_lon": "noise",
    "health_condition": "hash",
    "purchase_amount": "noise",
    "churned": "keep",
}
# Explication de mes choix : 
# j'ai choisi de supprimer les colonnes "id", "full_name", "gender", "country", "city", et "salary_band" car elles peuvent potentiellement identifier les individus.
# j'ai décidé de hash toutes les données personnelles et sensibles pour protéger la vie privée des individus.
# J'ai choisi de bin l'âge pour réduire la précision tout en conservant une certaine utilité analytique.
# J'ai opté pour le bruit sur les coordonnées GPS et le montant des achats pour préserver la confidentialité tout en permettant des analyses statistiques.
# Les colonnes non-identifiantes comme "churned" sont conservées telles quelles pour l'analyse.
print("Plan de protection des données :")
print(plan_protection)

Plan de protection des données :
{'id': 'drop', 'full_name': 'drop', 'email': 'hash', 'phone': 'hash', 'age': 'bin', 'gender': 'drop', 'country': 'drop', 'city': 'drop', 'salary_band': 'drop', 'ip_address': 'hash', 'gps_lat': 'noise', 'gps_lon': 'noise', 'health_condition': 'hash', 'purchase_amount': 'noise', 'churned': 'keep'}


## 6) Appliquer les transformations et générer le **livrable** (A compléter selon (5))
- Le **livrable** liste : colonne → catégorie → mesure de protection.
- Un **dataset transformé** est exporté.

In [11]:
def apply_plan(df_in: pd.DataFrame, classif_df: pd.DataFrame, plan: dict):
    df_out = df_in.copy()
    records = []
    for _, row in classif_df.iterrows():
        col = row["column"]
        cat = row.get("final_category", row.get("proposed_category", "non-identifiantes"))
        action = plan.get(col, "keep")
        records.append({"column": col, "category": cat, "protection": action})

        if action == "drop":
            if col in df_out.columns:
                df_out = df_out.drop(columns=[col])
        elif action == "hash":
            if col in df_out.columns:
                df_out[col] = sha256_hash_series(df_out[col], salt="CHANGE_ME_SALT")
        elif action == "bin":
            if col in df_out.columns:
                df_out[col] = bin_numeric(df_out[col], bins=6).astype(str)
        elif action == "noise":
            if col in df_out.columns:
                df_out[col] = noise_numeric(df_out[col], scale=0.05)
        elif action == "keep":
            pass
        else:
            print(f"[Info] Action inconnue pour {col}: {action}")
    livrable = pd.DataFrame(records)
    return df_out, livrable

# TODO : construire classif_df, puis :
df_transformed, livrable_df = apply_plan(df, classif_df, plan_protection)
display(livrable_df.head())
display(df_transformed.head())

,column,category,protection
0,id,quasi-identifiantes,drop
1,full_name,personnelle,drop
2,email,personnelle,hash
3,phone,personnelle,hash
4,age,personnelle,bin


,email,phone,age,ip_address,gps_lat,gps_lon,health_condition,purchase_amount,churned
0,a65e0c7dd001cd9188979a1851f3f44e3fd4f58e0d605a...,e2fa186a356b744d97b5de8333c342f3e01a4aaf23ad99...,"(38.0, 49.0]",817871f58f75bf716acfd40779b2aa56afbda599e40736...,48.887,2.367,64a63e472e2a91a8af878999101062f5f9dc301dc546fe...,181.846,0
1,38baec82b3248be8f1315a653a47c1efd884d65d041f3c...,dd850b9a6cd1a1bd4c755299428029587fc5f978232bb0...,"(15.999, 28.0]",bd958ce09e21127f98855b762f76615d7144ae6b581968...,48.712,2.354,64a63e472e2a91a8af878999101062f5f9dc301dc546fe...,93.116,0
2,d3dc06bbb0b9b16b745753a9ff048cc152d72b5a349e74...,f0777ddc14b4e2063595d6039d79245f452e3334f62a66...,"(72.5, 84.0]",17f3d0df624481fb276b95cc34b192ececffbca77cc5ae...,48.869,2.547,64a63e472e2a91a8af878999101062f5f9dc301dc546fe...,101.030,0
3,4f48db1af76ccda7d11e2ae3579291d920a10219eed479...,db62aa261d1b7b999ec59a75f26dfe449d275d875c3a19...,"(49.0, 62.0]",a4a220d5b50e7cef16389651b142515fbf6c2c5fa2a338...,48.824,2.201,0e11e91ca499ad89a0b06f1ba87c00b1ccadd0f3c0fcd0...,56.473,0
4,89e6fbdd29441a3db335c9a86e4225f8c82cd094d7cd7a...,bd96057d0945ed70c27c75c3fb0c02b2b87b68d89a5ec9...,"(28.0, 38.0]",bc040ecccf531f74773b6abf4a5aa97cbf2dfc722bcc94...,48.866,2.481,0e11e91ca499ad89a0b06f1ba87c00b1ccadd0f3c0fcd0...,89.693,1


## 7) Exporter (CSV)
- `livrable_cartographie.csv` : colonne → catégorie → mesure (+ params)
- `dataset_anonymized.csv` : dataset transformé


In [12]:
out_dir = Path("./outputs")
out_dir.mkdir(parents=True, exist_ok=True)

livrable_path = out_dir / "livrable_cartographie.csv"
data_out_path = out_dir / "dataset_anonymized.csv"

livrable_df.to_csv(livrable_path, index=False)
df_transformed.to_csv(data_out_path, index=False)

print("Fichiers exportés :")
print(" -", livrable_path.resolve())
print(" -", data_out_path.resolve())

Fichiers exportés :
 - C:\Users\wamba\Desktop\SCHOOL-DOC\2025-2027\COURS\DATA IA ML\PROTECTION_RGPD\outputs\livrable_cartographie.csv
 - C:\Users\wamba\Desktop\SCHOOL-DOC\2025-2027\COURS\DATA IA ML\PROTECTION_RGPD\outputs\dataset_anonymized.csv
